## Create a AOI geometry
For this experiment, we will use a point geometry to select a single tile. However, the process is inteneded to be able to accept polygon geometries as well to select multiple tiles.

The tile of interest is 15TXN689290 in the USGS_MN_RainyLake_1_2020 workunit. The tile contains the intersection of County Road 7 & County Road 44 west of Cascade River State Park.

In [1]:
from shapely import Point
import geopandas

point_of_interest = Point(-90.47416614755436, 47.738145812431185)  # EPSG:4326
input_gdf: geopandas.GeoDataFrame = geopandas.GeoDataFrame(
    geometry=[point_of_interest], crs="EPSG:4326"
)
input_gdf

,geometry
0,POINT (-90.47417 47.73815)


In [2]:
# Disable scroll wheel zoom so that the map doesn't zoom while scrolling through the notebook
input_gdf.explore(map_kwds={"scrollWheelZoom": False})

## Select intersecting tile(s)
Here we will intersect the input geometry with the tile index to select the tile of interest. We will use a `WHERE` clause to limit the intersection search to just the MN_RainyLake_2020 workunit.

In [3]:
from pathlib import Path


tiles_kwargs = {
    "filename": Path("../data/interim/tile_index.gpkg").resolve(),
    "layer": "tile_index",
    "mask": input_gdf.to_crs("EPSG:6344"),
    "where": "workunit='MN_RainyLake_1_2020'",
}

tiles: geopandas.GeoDataFrame = geopandas.read_file(**tiles_kwargs)
tiles

ERROR 1: PROJ: proj_create_from_database: Open of /home/dpower/mambaforge/envs/culvert-vision/share/proj failed


,name,workunit_id,workunit,geometry
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5..."


In [4]:
# Disable scroll wheel zoom so that the map doesn't zoom while scrolling through the notebook
tiles.explore(map_kwds={"scrollWheelZoom": False})

## Enrich the selected tile(s)
Next we will join workunit info to the tile(s) such as the href to the Entwine Point Tile (ept).

In [5]:
# The below successfully returns the workunit polygon,
# but it does not have the ept information we need
workunit_kwargs = {
    "filename": Path("../data/interim/mn_wesm.gpkg").resolve(),
    "layer": "mn_wesm",
    "where": "workunit='MN_RainyLake_1_2020'",
}
workunit_gdf: geopandas.GeoDataFrame = geopandas.read_file(**workunit_kwargs)
workunit_gdf

,workunit,workunit_id,project,project_id,collect_start,collect_end,ql,spec,p_method,dem_gsd_meters,...,sourcedem_category,sourcedem_reason,onemeter_category,onemeter_reason,seamless_category,seamless_reason,lpc_link,sourcedem_link,metadata_link,geometry
0,MN_RainyLake_1_2020,197389,MN_RainyLake_2020_B20,197392,2021-04-16,2021-05-17,QL 1,USGS Lidar Base Specification 2.1,linear-mode lidar,0.5,...,Meets,Meets 3DEP source DEM requirements,Meets,Meets 3DEP 1-m DEM requirements,Meets,Meets 3DEP seamless DEM requirements,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,http://prd-tnm.s3.amazonaws.com/index.html?pre...,http://prd-tnm.s3.amazonaws.com/index.html?pre...,"MULTIPOLYGON (((-92.81017 48.58193, -92.80842 ..."


In [6]:
# Copied from ../config/mn_rainylake_1_2020.json
ept_json_href = (
    "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/MN_RainyLake_1_2020/ept.json"
)
ept_json_href

'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/MN_RainyLake_1_2020/ept.json'

In [7]:
enriched_tiles = tiles.assign(ept_json_href=ept_json_href)
enriched_tiles

,name,workunit_id,workunit,geometry,ept_json_href
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...


## Calculate PDAL Pipeline parameters
Next we will calculate the unique parameters required to produce a PDAL pipeline definition that creates a DEM GeoTiff from streamed ept points. The parameters required for each stage are as follows.

**readers.ept** - Reads source
 - URL to ept.json
 - Buffered tile polygon expressed as WKT

**filters.range** - Filters for only ground classified points
 - None

**"filters.reprojection** - Projects the points to match the tile CRS
 - Output CRS expressed as an EPSG string ("EPSG:6344")

**filters.delaunay** - Create ground mesh
 - None

**filters.faceraster** - Calculate the raster cell values
 - Resolution in meters
 - Width of output raster (number of cells in the x dimension)
 - Height of output raster (number of cells in the y dimension)
 - Origin x coordinate (lower left)
 - Origin y coordinate (lower left)

**writers.raster** - Write the raster to disk
 - Output filename

In [8]:
# First we make a deep copy of the enriched_tiles so that we can make inplace
# modifications without overwriting the previous step
pipeline_params: geopandas.GeoDataFrame = enriched_tiles.copy(deep=True)
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...


In [9]:
# Create buffered tile WKT
buffer_dist = 5  # meters
pipeline_params["buffered_wkt"] = (
    pipeline_params.buffer(distance=buffer_dist).to_crs("EPSG:3857").to_wkt()
)
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href,buffered_wkt
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-10070597.571284 6062943.534161, -10..."


In [10]:
# Assign the EPSG string of the tiles to a field
pipeline_params["out_srs"] = pipeline_params.crs.srs
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href,buffered_wkt,out_srs
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-10070597.571284 6062943.534161, -10...",EPSG:6344


In [38]:
# Assign fields for the filters.faceraster parameters
resolution = 0.30  # meters

pipeline_params["resolution"] = resolution
pipeline_params["width"] = (
    (
        pipeline_params.geometry.bounds["maxx"]
        - pipeline_params.geometry.bounds["minx"]
        + resolution
    )
    / resolution
).astype(int)
pipeline_params["height"] = (
    (
        pipeline_params.geometry.bounds["maxy"]
        - pipeline_params.geometry.bounds["miny"]
        + resolution
    )
    / resolution
).astype(int)
pipeline_params["origin_x"] = pipeline_params.geometry.bounds["minx"]
pipeline_params["origin_y"] = pipeline_params.geometry.bounds["miny"]
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href,buffered_wkt,out_srs,resolution,width,height,origin_x,origin_y,output_file,pipeline_array
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-10070597.571284 6062943.534161, -10...",EPSG:6344,0.3,3334,3334,689000.0,5290000.0,../data/processed/15TXN689290.tif,"[{'tag': 'read_data', 'type': 'readers.ept', '..."


In [43]:
# Assign the output_file field
output_dir = Path("../data/processed").resolve()
pipeline_params["output_file"] = "../data/processed/" + pipeline_params["name"] + "_1ft.tif"
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href,buffered_wkt,out_srs,resolution,width,height,origin_x,origin_y,output_file,pipeline_array
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-10070597.571284 6062943.534161, -10...",EPSG:6344,0.3,3334,3334,689000.0,5290000.0,../data/processed/15TXN689290_1ft.tif,"[{'tag': 'read_data', 'type': 'readers.ept', '..."


In [44]:
def create_pipeline_array(
    ept_json_href: str,
    buffered_tile_wkt: str,
    out_srs: str,
    resolution: float,
    width: int,
    height: int,
    origin_x: float,
    origin_y: float,
    output_file: Path,
) -> list[dict[str, str]]:
    return [
        {
            "tag": "read_data",
            "type": "readers.ept",
            "filename": ept_json_href,
            "polygon": buffered_tile_wkt,
        },
        {
            "tag": "ground_only",
            "type": "filters.range",
            "limits": "Classification[2:2]",
        },
        {
            "tag": "reproject",
            "type": "filters.reprojection",
            "out_srs": out_srs,
        },
        {
            "tag": "create_mesh",
            "type": "filters.delaunay",
        },
        {
            "tag": "define_raster",
            "type": "filters.faceraster",
            "resolution": resolution,
            "width": int(width),
            "height": int(height),
            "origin_x": origin_x,
            "origin_y": origin_y,
        },
        {
            "tag": "write_raster",
            "type": "writers.raster",
            "filename": str(Path(output_file).resolve()),
            "gdaldriver": "GTiff",
            "gdalopts": "COMPRESS=DEFLATE",
            "data_type": "float32",
            "nodata": -999999,
        },
    ]

In [45]:
# Assign the pipeline array field
pipeline_params["pipeline_array"] = pipeline_params.apply(
    lambda x: create_pipeline_array(
        x.ept_json_href,
        x.buffered_wkt,
        x.out_srs,
        x.resolution,
        x.width,
        x.height,
        x.origin_x,
        x.origin_y,
        x.output_file,
    ),
    axis=1,
)
pipeline_params

,name,workunit_id,workunit,geometry,ept_json_href,buffered_wkt,out_srs,resolution,width,height,origin_x,origin_y,output_file,pipeline_array
0,15TXN689290,197389,MN_RainyLake_1_2020,"POLYGON ((690000.000 5290000.000, 689000.000 5...",https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-10070597.571284 6062943.534161, -10...",EPSG:6344,0.3,3334,3334,689000.0,5290000.0,../data/processed/15TXN689290_1ft.tif,"[{'tag': 'read_data', 'type': 'readers.ept', '..."


In [46]:
pipeline_params.at[0, "pipeline_array"]

[{'tag': 'read_data',
  'type': 'readers.ept',
  'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/MN_RainyLake_1_2020/ept.json',
  'polygon': 'POLYGON ((-10070597.571284 6062943.534161, -10070597.630806 6062942.806185, -10070597.761179 6062942.087567, -10070597.961148 6062941.385227, -10070598.228785 6062940.705929, -10070598.561514 6062940.056215, -10070598.95613 6062939.442342, -10070599.408834 6062938.870222, -10070599.915264 6062938.345365, -10070600.470544 6062937.872826, -10070601.069327 6062937.457155, -10070601.705846 6062937.102355, -10070602.37397 6062936.811844, -10070603.067265 6062936.588419, -10070603.779055 6062936.434231, -10070604.502485 6062936.350766, -10070605.230587 6062936.338828, -10072088.51854 6062984.942329, -10072089.244321 6062985.001908, -10072089.960784 6062985.132555, -10072090.661031 6062985.333013, -10072091.338318 6062985.601352, -10072091.986121 6062985.934986, -10072092.598203 6062986.330703, -10072093.168668 6062986.784691, -1007209

## Run the PDAL Pipeline
Since we are only using one tile in this experiment, we will access the pipeline_array field directly. In the production version, this will need to handle multiple extracting and executing multiple pipelines.

In [47]:
import pdal
import json


pipeline = pdal.Pipeline(json.dumps(pipeline_params.at[0, "pipeline_array"]))
count = pipeline.execute()